# Epipolar Geometry and Fundamental matrix

In [12]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

## Chessboard points as corresponding points

In [13]:
# Look at img0.png and img1.png and insert appropriate CHESSBOARD_SIZE
CHESSBOARD_SIZE=(9,6)

In [14]:
# Load images
img0=cv2.imread("img0.png")
img1=cv2.imread("img1.png")
img0Gray=cv2.cvtColor(img0,cv2.COLOR_BGR2GRAY)
img1Gray=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img0GrayEQ= cv2.equalizeHist(img0Gray)
img1GrayEQ= cv2.equalizeHist(img1Gray)
#Detecting chessboard corner points points0, points1 with cv2.findChessboardCorners(...) from img0 and img1.
(ret0,points0) = cv2.findChessboardCorners(img0GrayEQ,CHESSBOARD_SIZE,cv2.CALIB_CB_ADAPTIVE_THRESH)
(ret1,points1) = cv2.findChessboardCorners(img1GrayEQ,CHESSBOARD_SIZE,cv2.CALIB_CB_ADAPTIVE_THRESH)


## Estimating fundamental matrix using direct linear transformation.

In [10]:
# Estimating fundamental matrix
L=np.ones((len(points0),9))

# TODO1 Building left matrix
for i in range(0,len(points0)):
     p0=points0[i][0]
    p1=points1[i][0]
    L[i,0]=p1[0]*p0[0]#x1x0
    L[i,1]=p1[0]*p0[1]#x1y0
    L[i,2]=p1[0]#x1
    L[i,3]=p1[1]*p0[0]#y1x0
    L[i,4]=p1[1]*p0[1]#y1y0
    L[i,5]=p1[1]#y1
    L[i,6]=p0[0]#x0
    L[i,7]=p0[1]#y0
# END_OF TODO1

# TODO2 SVD from left matrix
u,s,vt=np.linalg.svd(L)
# END_OF TODO2

# TODO3 Get fundametnal matrix
f_vector=vt[8]
F=np.reshape(f_vector, (3,3)) # reshape
# END_OF TODO3
print("Fundamental matrix:")
print(F)

## Epipoles and epipolar lines

In [16]:
# Epipolar lines and epipole in img1
# TODO4 calculate epipole e1 in img1. 
#Hint: Epipol in Bild 2 wird berechnet durch die SVD von F.transponiert, dann ist der Epipol der normalisierte Vektor der V-Matrix der zum kleinsten Singulärwert gehört 
uft,sft,vft_t=np.linalg.svd(F.transpose())
e1=vft_t[2]
e1/=e1[2]
# END_OF TODO4

l1s=[]
#Calculate epipolar lines l1s corresponding to all points0
for i in range(0,len(points0)):
    p0=points0[i][0]
    p0homogen=np.array([ [p0[0]] , [p0[1]] ,[ 1]])
    # TODO5 calculate epipolar lines l1s corresponding to points0
    # Hint: line = dot_product (F . p0)
    l1s.append(None)
    # END_OF TODO5

## Displaying

In [9]:
imgWidth=img0.shape[1]
imgHConcat=cv2.hconcat([img0,img1])
t=imgRBG=cv2.cvtColor(imgHConcat,cv2.COLOR_BGR2RGB)
for i in range(0,len(points0)):
    img=imgHConcat.copy()
    # Mark point on img0
    p0=points0[i][0]
    cv2.circle(img,(int(p0[0]),int(p0[1])),3,(0,0,255),30)
    # Mark line on img1
    l1=l1s[i]
    left=(imgWidth,int(-l1[2]/l1[1]))
    right=(imgWidth*2,int((-l1[2]-l1[0]*imgWidth)/l1[1]))
    cv2.line(img,left,right,(0,0,255),5)
    # Mark epipole on img1
    cv2.circle(img,(imgWidth+int(e1[0]),int(e1[1])),4,(255,0,0),30)
    plt.imshow(img)
    plt.show()